In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import datetime
import matplotlib.pyplot as plt
import contextily as cx

## 读数据

In [2]:
path = 'process data/'
df = pd.read_csv(path + 'dataset_3_raw.csv')
df

,device_id,latitude,longitude,date
0,f664b4d6-5809-413e-b49b-3605d0515503,22.338962,114.207170,2023-09-04 07:41:03
1,e9017f7c-c52f-4a6a-926e-b1c0976b1248,22.288587,114.195222,2023-09-05 11:16:02
2,f6a9d8b6-5e2c-488f-9ee7-f58e97e5d5de,22.340000,114.200000,2023-09-14 11:21:56
3,ff22c280-126f-600f-214c-0ab0411394db,22.370519,114.132008,2023-09-01 18:15:22
4,ff22c280-126f-600f-214c-0ab0411394db,22.368826,114.118049,2023-09-01 20:50:06
...,...,...,...,...
3129760,b58ded3e-5a73-47e9-8af1-58ba760ac6d0,22.395124,113.974522,2023-09-06 06:47:37
3129761,b58ded3e-5a73-47e9-8af1-58ba760ac6d0,22.395138,113.974516,2023-09-06 07:15:54
3129762,b5933b4e-0e52-4398-8253-1f27bb4bec07,22.273868,114.187822,2023-09-15 03:06:44
3129763,b460a842-0630-4ebf-963b-6285b81f5574,22.310000,114.230000,2023-09-14 13:08:46


In [3]:
import datetime
df['date'] = pd.to_datetime(df['date'])

In [9]:
## 没有home经纬度的数据中，每个整点的数据量
df['date'].dt.hour.value_counts().sort_index()

0      61483
1      51411
2      55326
3      57644
4      45765
5      47723
6      85847
7     181680
8      91768
9     171672
10    204694
11    217428
12    239160
13    249172
14    218030
15    208176
16    177224
17    149450
18    148392
19    100611
20     88213
21     99076
22     99682
23     80138
Name: date, dtype: int64

In [4]:
df_15 = df[(df['date'].dt.hour >= 1) & (df['date'].dt.hour <= 5)] # 凌晨1：00至3：59的数据
df_15 = df_15[['device_id','latitude','longitude']]
df_15

,device_id,latitude,longitude
36,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,22.445384,114.035978
37,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,22.445352,114.035938
80,eb99111a-261c-4456-8da3-9f14c043b571,22.448940,113.998410
81,eb99111a-261c-4456-8da3-9f14c043b571,22.448940,113.998430
82,eb99111a-261c-4456-8da3-9f14c043b571,22.448920,113.998430
...,...,...,...
3129717,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424879,114.229265
3129718,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424917,114.229258
3129719,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424910,114.229247
3129720,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,22.424897,114.229254


In [5]:
device_cnt = np.size(df_15['device_id'].unique())
device_cnt
## 可以推测住址的用户有 44258 个

44258

In [6]:
## 转换为 GeoDataFrame
geometry = gpd.points_from_xy(df_15['longitude'], df_15['latitude'])
location_lat_lon_gdf = gpd.GeoDataFrame(df_15, geometry=geometry)
location_lat_lon_gdf.crs = 'EPSG:4326'
location_lat_lon_gdf = location_lat_lon_gdf.drop(['longitude','latitude'], axis=1)
location_lat_lon_gdf

,device_id,geometry
36,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,POINT (114.03598 22.44538)
37,e60fbeb4-7e78-4985-bcc3-09ff874cb7e3,POINT (114.03594 22.44535)
80,eb99111a-261c-4456-8da3-9f14c043b571,POINT (113.99841 22.44894)
81,eb99111a-261c-4456-8da3-9f14c043b571,POINT (113.99843 22.44894)
82,eb99111a-261c-4456-8da3-9f14c043b571,POINT (113.99843 22.44892)
...,...,...
3129717,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22927 22.42488)
3129718,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22926 22.42492)
3129719,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22925 22.42491)
3129720,afc663ee-9b82-4bfc-a45f-6cce84cb35b2,POINT (114.22925 22.42490)


## 读building shape file

In [7]:
import fiona

structure_path = 'Building_SHP/BUILDING_STRUCTURE.shp'
with fiona.open(structure_path) as f:
    shapefile = gpd.GeoDataFrame.from_features(f, crs=f.crs)
shapefile.to_crs(epsg=4326, inplace=True)
building_shp = shapefile[shapefile['BUILDING00'] == 'T']
building_shp = building_shp[['BUILDINGST', 'geometry']]
# 增加缓冲区
building_shp['geometry'] = building_shp.geometry.buffer(distance=0.0001)
building_shp

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_4452\3223849686.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  building_shp['geometry'] = building_shp.geometry.buffer(distance=0.0001)


,BUILDINGST,geometry
0,243561,"POLYGON ((113.84081 22.21859, 113.84080 22.218..."
1,243562,"POLYGON ((113.84093 22.21873, 113.84094 22.218..."
2,243563,"POLYGON ((113.84090 22.21857, 113.84089 22.218..."
3,243564,"POLYGON ((113.84087 22.21873, 113.84087 22.218..."
4,243565,"POLYGON ((113.84100 22.21861, 113.84099 22.218..."
...,...,...
340747,403798,"POLYGON ((114.43645 22.54009, 114.43646 22.540..."
340748,4963,"POLYGON ((114.43643 22.54126, 114.43644 22.541..."
340749,4952,"POLYGON ((114.43677 22.54114, 114.43676 22.541..."
340750,5040,"POLYGON ((114.43675 22.54040, 114.43670 22.540..."


## 关联 building_shp 和 location_lat_lon_gdf  #merged

In [8]:
merged = gpd.sjoin(building_shp, location_lat_lon_gdf, how='inner', predicate='intersects')
# merged = merged.drop('index_right', axis=1)
merged

,BUILDINGST,geometry,index_right,device_id
310,246850,"POLYGON ((113.85967 22.25511, 113.85967 22.255...",2818976,2b4fd610-a53b-4859-a1fd-d65686bb9145
310,246850,"POLYGON ((113.85967 22.25511, 113.85967 22.255...",2818977,2b4fd610-a53b-4859-a1fd-d65686bb9145
494,336293,"POLYGON ((113.86179 22.25353, 113.86179 22.253...",2584067,cb70e6e9-b2ce-6a70-becb-b807e32c87de
534,190039,"POLYGON ((113.86222 22.25387, 113.86223 22.253...",2584067,cb70e6e9-b2ce-6a70-becb-b807e32c87de
537,336298,"POLYGON ((113.86237 22.25388, 113.86238 22.253...",2584067,cb70e6e9-b2ce-6a70-becb-b807e32c87de
...,...,...,...,...
339575,352677,"POLYGON ((114.33265 22.44002, 114.33265 22.440...",1916134,74243407-7daa-4a8f-82dc-e8baf8f802c1
339575,352677,"POLYGON ((114.33265 22.44002, 114.33265 22.440...",1916133,74243407-7daa-4a8f-82dc-e8baf8f802c1
339922,42867,"POLYGON ((114.35851 22.46744, 114.35851 22.467...",2031642,c8752678-f044-4c71-8ce5-9c48443f7507
339922,42867,"POLYGON ((114.35851 22.46744, 114.35851 22.467...",2031644,c8752678-f044-4c71-8ce5-9c48443f7507


In [12]:
np.size(merged['device_id'].unique())

13782

In [13]:
device_building_loc_df = merged[['device_id','BUILDINGST']]
device_building_loc_df

,device_id,BUILDINGST
310,2b4fd610-a53b-4859-a1fd-d65686bb9145,246850
310,2b4fd610-a53b-4859-a1fd-d65686bb9145,246850
494,cb70e6e9-b2ce-6a70-becb-b807e32c87de,336293
534,cb70e6e9-b2ce-6a70-becb-b807e32c87de,190039
537,cb70e6e9-b2ce-6a70-becb-b807e32c87de,336298
...,...,...
339575,74243407-7daa-4a8f-82dc-e8baf8f802c1,352677
339575,74243407-7daa-4a8f-82dc-e8baf8f802c1,352677
339922,c8752678-f044-4c71-8ce5-9c48443f7507,42867
339922,c8752678-f044-4c71-8ce5-9c48443f7507,42867


## 推测住址

In [14]:
## 计算每个 device_id 的每个 BUILDINGST 出现的次数
counts = device_building_loc_df.groupby(['device_id','BUILDINGST']).size()
counts

device_id                             BUILDINGST
0002d2d4-b519-4255-9167-b6c7a9fa9f5b  14173          1
                                      14851          5
                                      14855          5
                                      14860          5
                                      14863          5
                                                    ..
fff6fd4a-9773-44e7-b16a-6ddfdf4429f0  98848          1
fff7c7a2-9b43-4fb2-ab5f-06c479d2b996  134798         1
fffc482d-4e69-48b6-a765-9c4468f60d90  176938         1
                                      181409        16
fffc99eb-689e-4108-a41f-399d21a78a1b  98848          6
Length: 39212, dtype: int64

In [15]:
## 保存 device_id 和 这个BUILDINGST出现次数最多的 BUILDINGST
max_BUILDINGST = counts.groupby('device_id').idxmax().apply(lambda x: x[1])
max_BUILDINGST

device_id
0002d2d4-b519-4255-9167-b6c7a9fa9f5b     14851
000976e6-c4ca-662c-1022-8bafbe934242    137369
0009c58a-189f-4700-b2f4-3bda2b6de6c4    160930
000c8b3f-fc84-63b7-13b4-a674302c34b7    165947
0014bb13-b820-4646-accb-08f9877d4c9c     42988
                                         ...  
fff69326-c5b2-4e65-a83f-38cc351d87ba    345867
fff6fd4a-9773-44e7-b16a-6ddfdf4429f0     98848
fff7c7a2-9b43-4fb2-ab5f-06c479d2b996    134798
fffc482d-4e69-48b6-a765-9c4468f60d90    181409
fffc99eb-689e-4108-a41f-399d21a78a1b     98848
Length: 13782, dtype: int64

In [18]:
# f_counts = counts[counts > 1] ## 剔除只出现过一次的building
# f_counts.groupby('device_id').idxmax().apply(lambda x: x[1])

device_id
0002d2d4-b519-4255-9167-b6c7a9fa9f5b     14851
000976e6-c4ca-662c-1022-8bafbe934242    137369
00151bcd-fe23-4cb8-a49c-1dbe786be2be    196955
001ba349-ab7b-4c7c-b796-fc3bd6751660    169262
0020bd39-a85b-489f-825c-ff9d6b0cf92a    209162
                                         ...  
ffbe74d6-fea8-46d0-adc7-229938ad6daf    127933
ffc16aa5-fae1-4923-aca6-0bf04aca99c8     17755
ffcf767c-1eaa-44eb-b6ae-3b74f6030fb6    126178
fffc482d-4e69-48b6-a765-9c4468f60d90    181409
fffc99eb-689e-4108-a41f-399d21a78a1b     98848
Length: 7119, dtype: int64

In [16]:
location_building_pred_df = pd.DataFrame({'device_id': max_BUILDINGST.index, 'BUILDINGST': max_BUILDINGST.values})
location_building_pred_df

## 44258 个用户中， 成功推测出住址的有 13782 个

,device_id,BUILDINGST
0,0002d2d4-b519-4255-9167-b6c7a9fa9f5b,14851
1,000976e6-c4ca-662c-1022-8bafbe934242,137369
2,0009c58a-189f-4700-b2f4-3bda2b6de6c4,160930
3,000c8b3f-fc84-63b7-13b4-a674302c34b7,165947
4,0014bb13-b820-4646-accb-08f9877d4c9c,42988
...,...,...
13777,fff69326-c5b2-4e65-a83f-38cc351d87ba,345867
13778,fff6fd4a-9773-44e7-b16a-6ddfdf4429f0,98848
13779,fff7c7a2-9b43-4fb2-ab5f-06c479d2b996,134798
13780,fffc482d-4e69-48b6-a765-9c4468f60d90,181409


In [17]:
## 保存数据：device_building_location
location_building_pred_df.to_csv(path + "dataset3/device_building_location.csv", index=False)